# Topic Modeling with SciKit Learn

In this notebook we create a topic model from our corpus using SciKit Learn's text feature extraction library. We'll save our results and then use another notebook to explore the results.

# Set Up

## Imports

In [1]:
import pandas as pd
import numpy as np
from lib import tapi

## Configuration

In [2]:
tapi.list_corpora()

['airbnb',
 'anphoblacht',
 'arxiv',
 'covid19',
 'jstor_hyperparameter',
 'novels',
 'okcupid',
 'tamilnet',
 'winereviews',
 'yelp',
 'zuboff']

In [3]:
data_prefix = 'winereviews'

In [4]:
db = tapi.Edition(data_prefix)

## Parameters

In [5]:
n_terms = 4000      # Vocabulary size
ngram_range = (1,4) # ngram min and max lengths
n_topics = 20       # Number of topics
max_iter = 5        # Number of iterations for topic model

## Create Tables Object

These tables constitute a "digital critical edition."

# Import Corpus Data

We import a corpus in our standard format

In [6]:
corpus = db.get_corpus()

## Inspect contents

In [7]:
corpus

,doc_key,doc_title,doc_label,doc_province,doc_points,doc_price,doc_content,doc_original,doc_variety,doc_taster,doc_place
doc_id,,,,,,,,,,,
0,86023,Lange 2011 Three Hills Cuvée Pinot Noir (Willa...,US,Oregon,88,40.0,"A tart, astringent Pinot, it needs a bit more ...","A tart, astringent Pinot, it needs a bit more ...",Pinot Noir,Paul Gregutt,US Oregon Willamette Valley Willamette Valley
1,45852,Finca Casa Lo Alto 2008 Reserva Red (Utiel-Req...,Spain,Levante,84,35.0,The cola and licorice aromas are candied and e...,The cola and licorice aromas are candied and e...,Red Blend,Michael Schachner,Spain Levante Utiel-Requena
2,32297,Plantagenet 2004 Omrah Cabernet Sauvignon (Wes...,Australia,Western Australia,88,15.0,"A good value, this starts off a little shaky t...","A good value, this starts off a little shaky t...",Cabernet Sauvignon,Joe Czerwinski,Australia Western Australia Western Australia
3,43293,Bougrier 2012 Rosé d'Anjou (Rosé) by Roger Voss,France,Loire Valley,84,13.0,"Typical, light and sweet rosé, fruity with bri...","Typical, light and sweet rosé, fruity with bri...",Rosé,Roger Voss,France Loire Valley Rosé d'Anjou
4,118523,Bolla 2007 Le Poiane (Valpolicella Classico S...,Italy,Veneto,87,14.0,If you aren't familiar with Ripasso (a hybrid ...,If you aren't familiar with Ripasso (a hybrid ...,"Corvina, Rondinella, Molinara",NaN,Italy Veneto Valpolicella Classico Superiore R...
...,...,...,...,...,...,...,...,...,...,...,...
9995,114715,Höpler 2014 Pannonica White (Burgenland) Panno...,Austria,Burgenland,88,11.0,Very slight hints of fresh pear peel on nose a...,Very slight hints of fresh pear peel on nose a...,White Blend,Anne Krebiehl MW,Austria Burgenland
9996,120801,Les Vignerons de Tavel 2016 Cuvée Royale (Tav...,France,Rhône Valley,90,17.0,A slightly confected cherry note on the nose i...,A slightly confected cherry note on the nose i...,Rosé,Joe Czerwinski,France Rhône Valley Tavel
9997,125926,Longboard 2010 Mavericks Goosechase Vineyard C...,US,California,92,65.0,This is a seriously good Cabernet. You can sta...,This is a seriously good Cabernet. You can sta...,Cabernet Sauvignon,NaN,US California Alexander Valley Sonoma


In [8]:
corpus.doc_content.sample(10).to_list()

['The complex, seductive nose promises a bit more than the palate delivers. Nonetheless, there is good complexity and a moderately long, if somewhat watery, finish. The slight residual sugar will make this a good choice with drier Asian dishes. Drink now.',
 'This opens with alluring scents of violet, rose, red berry and chopped herbs. The fresh, delicious palate doles out pure fruit flavors of black cherry, blackberry and raspberry while licorice and clove notes offer back up. Silky tannins and bright acidity provide the pliant framework.',
 'This attractive wine is full of ripe fruit, with soft tannins and only light acidity. It is developing well. Drink this wine from late 2017.',
 'The aromas of boysenberry, black pepper and roasting game are reminiscent of Syrah in this tasty bottling from brother-vintners David and William Vondrasek. The palate offers a lot of zing, followed by flavors of tobacco ash, olallieberry and peppery bacon.',
 'Picture-perfect coastal California Sauvigno

In [9]:
# corpus.head(10)

# Convert to Bag of Words 

ie. a __Count Vector Space__

We use Scikit Learn's CountVectorizer to convert our corpus of documents into a document-term vector space of word counts.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [11]:
count_engine = CountVectorizer(max_features=n_terms, stop_words='english', ngram_range=ngram_range)
count_model = count_engine.fit_transform(corpus.doc_content)

## Get Generated VOCAB

In [12]:
db.VOCAB = pd.DataFrame(count_engine.get_feature_names(), columns=['term_str'])
db.VOCAB = db.VOCAB.set_index('term_str')
db.VOCAB['ngram_len'] = None # To be added later

In [13]:
db.VOCAB.sample(10)

,ngram_len
term_str,
precise,None
substantial,None
petals,None
dark plum,None
plum fruit,None
possible,None
tart fruit,None
spicy oak,None
inviting,None


## Get Generated BOW

We do this just to show what the counter vectorizer produced. `DTM` stands for documet-term matrix. We convert this sparse matrix into a "thin" dataframe that keeps only terms with counts for each document. 

In [14]:
db.DTM = pd.DataFrame(count_model.toarray(), index=corpus.index, columns=db.VOCAB.index)
db.BOW = db.DTM.stack().to_frame('n')
db.BOW = db.BOW[db.BOW.n > 0]

In [15]:
db.DTM.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Columns: 4000 entries, 000 to zippy acidity
dtypes: int64(4000)
memory usage: 305.5 MB


In [16]:
db.BOW.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 267755 entries, (0, 'accented') to (9999, 'wine')
Columns: 1 entries, n to n
dtypes: int64(1)
memory usage: 3.5+ MB


## Compute TF-IDF

In [17]:
tfidf_engine = TfidfTransformer()
tfidf_model = tfidf_engine.fit_transform(count_model)

In [18]:
db.TFIDF = pd.DataFrame(tfidf_model.toarray(), index=corpus.index, columns=db.VOCAB.index)

In [19]:
db.BOW['tfidf'] = db.TFIDF.stack()

In [20]:
db.BOW

n     tfidf
doc_id term_str                
0      accented     1  0.212521
       astringent   1  0.219259
       barrel       1  0.183510
       bit          1  0.164003
       bottle       1  0.198999
...                ..       ...
9999   showing      1  0.160109
       stone        1  0.161591
       stone fruit  1  0.177589
       texture      1  0.118262
       wine         1  0.059593

[267755 rows x 2 columns]

## Add Features to VOCAB

In [21]:
db.VOCAB['ngram_len'] = db.VOCAB.apply(lambda x: len(x.name.split()), 1)
db.VOCAB['n'] = db.DTM.sum()
db.VOCAB['tfidf_mean'] = db.TFIDF.mean()

In [22]:
db.VOCAB

,ngram_len,n,tfidf_mean
term_str,,,
000,1,20,0.000502
000 cases,2,14,0.000366
10,1,141,0.002449
10 merlot,2,14,0.000324
10 years,2,30,0.000687
...,...,...,...
zin,1,59,0.001335
zinfandel,1,89,0.001958
zingy,1,19,0.000526


In [23]:
db.VOCAB.ngram_len.value_counts()

1    2099
2    1722
3     172
4       7
Name: ngram_len, dtype: int64

# Generate Topic Models

We run Scikit Learn's [LatentDirichletAllocation algorithm](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html#sklearn.decomposition.LatentDirichletAllocation) and extract the THETA and PHI tables.

In [24]:
from sklearn.decomposition import LatentDirichletAllocation as LDA, NMF

## Using LDA

In [25]:
lda_engine = LDA(n_components=n_topics, max_iter=max_iter, learning_offset=50., random_state=0)

### THETA

The Document-Term Matrix

In [26]:
db.THETA = pd.DataFrame(lda_engine.fit_transform(count_model), index=corpus.index)
db.THETA.index.name = 'doc_id'
db.THETA.columns.name = 'topic_id'

In [27]:
db.THETA.sample(20).style.background_gradient(axis=1)

topic_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
doc_id,,,,,,,,,,,,,,,,,,,,
2961,0.001563,0.001563,0.001563,0.927791,0.001563,0.001563,0.001563,0.001563,0.001563,0.001563,0.001563,0.001563,0.044084,0.001563,0.001563,0.001563,0.001563,0.001563,0.001563,0.001563
2362,0.001724,0.001724,0.001724,0.739240,0.229725,0.001724,0.001724,0.001724,0.001724,0.001724,0.001724,0.001724,0.001724,0.001724,0.001724,0.001724,0.001724,0.001724,0.001724,0.001724
3875,0.001220,0.001220,0.001220,0.001220,0.001220,0.001220,0.001220,0.001220,0.001220,0.001220,0.001220,0.342927,0.001220,0.001220,0.001220,0.001220,0.156608,0.001220,0.389551,0.091401
9770,0.112102,0.001515,0.001515,0.001515,0.001515,0.001515,0.001515,0.001515,0.001515,0.001515,0.001515,0.001515,0.001515,0.001515,0.001515,0.602776,0.001515,0.001515,0.001515,0.259364
9573,0.001667,0.146612,0.001667,0.001667,0.001667,0.001667,0.001667,0.001667,0.001667,0.001667,0.001667,0.001667,0.001667,0.001667,0.001667,0.001667,0.001667,0.001667,0.823388,0.001667
4817,0.002083,0.002083,0.002083,0.002083,0.640173,0.002083,0.002083,0.002083,0.002083,0.002083,0.202805,0.002083,0.002083,0.121606,0.002083,0.002083,0.002083,0.002083,0.002083,0.002083
8426,0.001471,0.001471,0.001471,0.120280,0.001471,0.001471,0.853250,0.001471,0.001471,0.001471,0.001471,0.001471,0.001471,0.001471,0.001471,0.001471,0.001471,0.001471,0.001471,0.001471
8384,0.002778,0.002778,0.002778,0.002778,0.002778,0.002778,0.002778,0.002778,0.002778,0.002778,0.002778,0.047403,0.002778,0.002778,0.902597,0.002778,0.002778,0.002778,0.002778,0.002778
9867,0.002500,0.002500,0.002500,0.002500,0.002500,0.770509,0.002500,0.002500,0.002500,0.002500,0.002500,0.184491,0.002500,0.002500,0.002500,0.002500,0.002500,0.002500,0.002500,0.002500


### PHI

In [28]:
db.PHI = pd.DataFrame(lda_engine.components_, columns=db.VOCAB.index)
db.PHI.index.name = 'topic_id'
db.PHI.columns.name  = 'term_str'

In [29]:
db.PHI.head().style.background_gradient()

### Create Topic Glosses

In [30]:
n_top_words = 7

In [31]:
db.TOPICS = db.PHI.stack()\
    .to_frame('weight')\
    .groupby('topic_id')\
    .apply(lambda x: x.weight.sort_values(ascending=False)\
               .head(n_top_words)\
               .reset_index()\
               .drop('topic_id',1)\
               .term_str)

In [32]:
db.TOPICS

term_str,0,1,2,3,4,5,6
topic_id,,,,,,,
0,nose,fruit,palate,shows,bottling,aromas,flavors
1,fruit,wine,flavors,stone,stone fruit,aromas,blend
2,ripe,acidity,fruit,wine,bright,flavors,drink
3,wine,drink,acidity,fruits,ripe,tannins,fruit
4,flavors,acidity,wine,finish,peach,lemon,palate
5,wine,fruit,acidity,blanc,flavors,texture,sauvignon blanc
6,palate,cherry,tannins,black,aromas,alongside,offers
7,fruit,flavors,finish,wine,black,fruit flavors,oak
8,wine,tannins,black,plum,flavors,berry,blackberry


In [33]:
db.TOPICS['topwords'] = db.TOPICS.apply(lambda x: str(x.name) + ' ' + ', '.join(x), 1)

In [34]:
db.TOPICS

term_str,0,1,2,3,4,5,6,topwords
topic_id,,,,,,,,
0,nose,fruit,palate,shows,bottling,aromas,flavors,"0 nose, fruit, palate, shows, bottling, aromas..."
1,fruit,wine,flavors,stone,stone fruit,aromas,blend,"1 fruit, wine, flavors, stone, stone fruit, ar..."
2,ripe,acidity,fruit,wine,bright,flavors,drink,"2 ripe, acidity, fruit, wine, bright, flavors,..."
3,wine,drink,acidity,fruits,ripe,tannins,fruit,"3 wine, drink, acidity, fruits, ripe, tannins,..."
4,flavors,acidity,wine,finish,peach,lemon,palate,"4 flavors, acidity, wine, finish, peach, lemon..."
5,wine,fruit,acidity,blanc,flavors,texture,sauvignon blanc,"5 wine, fruit, acidity, blanc, flavors, textur..."
6,palate,cherry,tannins,black,aromas,alongside,offers,"6 palate, cherry, tannins, black, aromas, alon..."
7,fruit,flavors,finish,wine,black,fruit flavors,oak,"7 fruit, flavors, finish, wine, black, fruit f..."
8,wine,tannins,black,plum,flavors,berry,blackberry,"8 wine, tannins, black, plum, flavors, berry, ..."


### Add Doc Weights

In [35]:
db.TOPICS['doc_weight_sum'] = db.THETA.sum()

In [36]:
db.TOPICS.iloc[:, 7:].sort_values('doc_weight_sum', ascending=False).style.bar()

term_str,topwords,doc_weight_sum
topic_id,,
3,"3 wine, drink, acidity, fruits, ripe, tannins, fruit",1328.340014
4,"4 flavors, acidity, wine, finish, peach, lemon, palate",1090.538970
19,"19 flavors, finish, aromas, berry, palate, plum, oak",709.035556
14,"14 cherry, flavors, wine, pinot, aromas, black, noir",607.757173
18,"18 wine, flavors, ripe, tannins, rich, fruit, drink",578.143584
15,"15 apple, palate, finish, citrus, aromas, green, fresh",562.286731
13,"13 wine, fruit, oak, cabernet, blend, tannins, black",555.768099
6,"6 palate, cherry, tannins, black, aromas, alongside, offers",538.424108
9,"9 fruit, aromas, palate, wine, white, finish, red",434.146039


## Using NMF

In [37]:
nmf_engine = NMF(n_components=n_topics, init='nndsvd', random_state=1, alpha=.1, l1_ratio=.5)

### THETA

In [38]:
db.THETA_NMF = pd.DataFrame(nmf_engine.fit_transform(tfidf_model), index=corpus.index)
db.THETA_NMF.columns.name = 'topic_id'

In [39]:
db.THETA_NMF.sample(20).style.background_gradient()

topic_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
doc_id,,,,,,,,,,,,,,,,,,,,
2785,0.005547,0.000000,0.041228,0.000000,0.000000,0.000000,0.000000,0.000000,0.007755,0.000000,0.000000,0.041234,0.000000,0.000000,0.016852,0.001047,0.000000,0.011034,0.000000,0.000000
4566,0.008660,0.000000,0.000000,0.000000,0.000000,0.000000,0.032959,0.013818,0.013173,0.017098,0.000000,0.020903,0.000000,0.000000,0.000000,0.000000,0.004238,0.000000,0.004657,0.000000
7593,0.000000,0.004820,0.000000,0.065595,0.000000,0.000000,0.005445,0.022535,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3815,0.000000,0.005092,0.000000,0.063965,0.000000,0.000000,0.000000,0.032341,0.000000,0.003259,0.000000,0.000000,0.000000,0.000000,0.000000,0.001396,0.000000,0.000000,0.000000,0.010452
8109,0.000000,0.001947,0.000000,0.000000,0.004549,0.000000,0.059631,0.000000,0.000000,0.000000,0.000000,0.000000,0.018439,0.000000,0.000000,0.000000,0.000000,0.006448,0.000000,0.000000
8710,0.000000,0.000000,0.009590,0.004817,0.000000,0.000000,0.000000,0.000000,0.000000,0.012871,0.000000,0.000000,0.000000,0.000000,0.034770,0.011254,0.000000,0.000830,0.000000,0.000000
7240,0.011448,0.000232,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018037,0.028367,0.000000,0.000000,0.000990,0.000000,0.000000,0.000000,0.008773,0.006281,0.000000,0.000000
7999,0.008449,0.000000,0.000000,0.023169,0.000000,0.000000,0.003115,0.005185,0.017220,0.006767,0.000000,0.000000,0.000679,0.006212,0.000000,0.000000,0.001078,0.000000,0.000000,0.000000
8020,0.028728,0.006478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009664,0.000000,0.001116,0.000000,0.000741,0.000000,0.000000,0.000000,0.000000,0.009147,0.001423,0.000000


### PHI

In [40]:
db.PHI_NMF = pd.DataFrame(nmf_engine.components_, columns=db.VOCAB.index)

In [41]:
db.PHI_NMF.index.name = 'topic_id'
db.PHI_NMF.columns.name = 'term_str'

In [42]:
db.PHI_NMF.T.head().style.background_gradient()

topic_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
term_str,,,,,,,,,,,,,,,,,,,,
000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002404,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
000 cases,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10,0.043333,0.000000,0.000000,0.055016,0.160916,0.000000,0.000000,0.022803,0.000000,0.000000,0.000000,0.000000,0.017203,0.000000,0.000000,0.007809,0.024557,0.000000,0.000000,0.000000
10 merlot,0.000000,0.000000,0.000000,0.004434,0.036964,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10 years,0.023723,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.044730,0.000000,0.000000,0.000000


### Topics

In [43]:
db.TOPICS_NMF = db.PHI_NMF.stack()\
    .to_frame('weight')\
    .groupby('topic_id')\
    .apply(lambda x: 
           x.weight.sort_values(ascending=False)\
               .head(n_top_words)\
               .reset_index()\
               .drop('topic_id',1)\
               .term_str)

In [44]:
db.TOPICS_NMF

term_str,0,1,2,3,4,5,6
topic_id,,,,,,,
0,wine,fruits,ripe,rich,drink,tannins,wood
1,crisp,light,acidity,fresh,bright,wine,fruity
2,lemon,lime,lemon lime,grapefruit,zest,riesling,orange
3,tannins,alongside,palate,aromas,cherry,offers,palate offers
4,cabernet,sauvignon,cabernet sauvignon,blend,merlot,franc,cabernet franc
5,red,red berry,red cherry,red fruit,red fruits,berry,red currant
6,berry,finish,plum,flavors,aromas,herbal,feels
7,black,black cherry,cherry,pepper,black pepper,blackberry,currant
8,fruit,fruit flavors,flavors,tropical fruit,black fruit,tropical,aromas


In [45]:
db.TOPICS_NMF['topwords'] = db.TOPICS_NMF.apply(lambda x: str(x.name) + ' ' + ', '.join(x), 1)

### Add Doc Weights

In [46]:
db.TOPICS_NMF['doc_weight_sum'] = db.THETA_NMF.sum()

In [47]:
db.TOPICS_NMF.iloc[:, 7:].sort_values('doc_weight_sum', ascending=False).style.bar()

term_str,topwords,doc_weight_sum
topic_id,,
0,"0 wine, fruits, ripe, rich, drink, tannins, wood",89.181919
6,"6 berry, finish, plum, flavors, aromas, herbal, feels",70.376952
3,"3 tannins, alongside, palate, aromas, cherry, offers, palate offers",67.308991
1,"1 crisp, light, acidity, fresh, bright, wine, fruity",65.148885
11,"11 nose, palate, notes, nose palate, finish, shows, bottling",64.259420
9,"9 sweet, flavors, vanilla, like, pineapple, honey, soft",60.523884
7,"7 black, black cherry, cherry, pepper, black pepper, blackberry, currant",57.334993
8,"8 fruit, fruit flavors, flavors, tropical fruit, black fruit, tropical, aromas",55.332295
15,"15 white, peach, white peach, stone, flower, citrus, stone fruit",49.729342


# Save the Model

## Keep Corpus Label Info

This is effectively the LIB table.

In [48]:
db.LABELS = corpus[set(corpus.columns.tolist()) - set(['doc_key', 'doc_content', 'doc_original'])]

## Save Tables

In [49]:
db.save_tables()

In [50]:
# See if it worked ...

!ls -l ./db/{data_prefix}*.csv

-rw-r--r--@ 1 rca2t1  staff   9347007 Jun 16 09:56 ./db/winereviews-BOW.csv
-rw-r--r--@ 1 rca2t1  staff  80091190 Jun 16 09:57 ./db/winereviews-DTM.csv
-rw-r--r--@ 1 rca2t1  staff   1832690 Jun 16 09:56 ./db/winereviews-LABELS.csv
-rw-r--r--@ 1 rca2t1  staff   1387059 Jun 16 09:57 ./db/winereviews-PHI.csv
-rw-r--r--@ 1 rca2t1  staff    544306 Jun 16 09:57 ./db/winereviews-PHI_NMF.csv
-rw-r--r--@ 1 rca2t1  staff   4322057 Jun 16 09:57 ./db/winereviews-THETA.csv
-rw-r--r--@ 1 rca2t1  staff   1967524 Jun 16 09:57 ./db/winereviews-THETA_NMF.csv
-rw-r--r--@ 1 rca2t1  staff      2611 Jun 16 09:56 ./db/winereviews-TOPICS.csv
-rw-r--r--@ 1 rca2t1  staff      3015 Jun 16 09:57 ./db/winereviews-TOPICS_NMF.csv
-rw-r--r--@ 1 rca2t1  staff    150779 Jun 16 09:56 ./db/winereviews-VOCAB.csv
